# Evaluate accuracy of the different algorithms

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bread.algo.lineage import LineageGuesserBudLum, LineageGuesserExpansionSpeed, LineageGuesserMinDistance, LineageGuesserMinTheta, accuracy
from bread.data import Lineage, Segmentation, Microscopy, BreadWarning
from glob import glob
import warnings
import pandas as pd

In [3]:
accs = {'BudLum': [], 'ExpansionSpeed': [], 'MinTheta': [], 'MinDistance': []}
colony_ids = [1, 2, 3, 4, 5]

warnings.simplefilter('ignore', BreadWarning)

for colony_id in colony_ids:
	print(f'computing acc colony {colony_id}')
	seg = Segmentation.from_h5(f'../data/colony{colony_id:03d}_segmentation.h5')
	gfp = Microscopy.from_tiff(f'../data/colony{colony_id:03d}_GFP.tif')
	lin_truth = Lineage.from_csv(f'../data/colony{colony_id:03d}_lineage.csv')

	print('[', end='')
	lin_pred_budlum = LineageGuesserBudLum(budneck_img=gfp, segmentation=seg).guess_lineage()
	print('-', end='')
	lin_pred_expspeed = LineageGuesserExpansionSpeed(segmentation=seg).guess_lineage()
	print('-', end='')
	lin_pred_mintheta = LineageGuesserMinTheta(segmentation=seg).guess_lineage()
	print('-', end='')
	lin_pred_mindist = LineageGuesserMinDistance(segmentation=seg).guess_lineage()
	print('-]')

	accs['BudLum'].append(accuracy(lin_truth, lin_pred_budlum, strict=False))
	accs['ExpansionSpeed'].append(accuracy(lin_truth, lin_pred_expspeed, strict=False))
	accs['MinTheta'].append(accuracy(lin_truth, lin_pred_mintheta, strict=False))
	accs['MinDistance'].append(accuracy(lin_truth, lin_pred_mindist, strict=False))

computing acc colony 1
[----]
computing acc colony 2
[----]
computing acc colony 3
[----]
computing acc colony 4
[----]
computing acc colony 5
[----]


In [4]:
accs_df = pd.DataFrame(accs, [f'colony {i}' for i in colony_ids], accs.keys())

In [5]:
accs_df

,BudLum,ExpansionSpeed,MinTheta,MinDistance
colony 1,1.000000,0.750000,0.800000,0.762500
colony 2,0.993243,0.806897,0.736486,0.743243
colony 3,0.989474,0.731183,0.684211,0.610526
colony 4,0.968000,0.650000,0.592000,0.656000
colony 5,1.000000,0.805556,0.694444,0.777778


In [6]:
accs_df.mean()

BudLum            0.990143
ExpansionSpeed    0.748727
MinTheta          0.701428
MinDistance       0.710009
dtype: float64